# Colorectal Cancer Prediction

**This notebook contains a full, reproducible pipeline.**

Open in Jupyter or Google Colab and run all cells to execute EDA, preprocessing, model training, and evaluation.

## 1. Setup
Import libraries and load dataset

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Load data
df = pd.read_csv('colorectal_cancer_prediction.csv')
df.head()


## 2. Target detection & EDA
Detect target column, show shape, distributions, and basic stats

In [ ]:

# Detect likely target column
possible_targets = ['target','label','diagnosis','class','status']
target_col = None
for c in df.columns:
    if c.lower() in possible_targets:
        target_col = c; break
if target_col is None:
    target_col = df.columns[-1]
    
y_raw = df[target_col].astype(str).str.lower()
y = y_raw.replace({'positive':1,'negative':0,'yes':1,'no':0,'1':1,'0':0,'malignant':1,'benign':0})
try:
    y = y.astype(int)
except Exception:
    y, _ = pd.factorize(y)
X = df.drop(columns=[target_col]).copy()

print('Dataset shape:', df.shape)
print('Target column detected:', target_col)
print('Class distribution (normalized):')
print(pd.Series(y).value_counts(normalize=True))

# Basic EDA plots
plt.figure()
pd.Series(y).value_counts().sort_index().plot(kind='bar')
plt.title('Target Distribution (0=Neg,1=Pos)'); plt.show()

numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
print('Number of numeric features:', len(numeric_features))
if numeric_features:
    col = numeric_features[0]
    plt.figure()
    X[col].dropna().plot(kind='hist', bins=30)
    plt.title(f'Distribution: {col}'); plt.show()
if len(numeric_features) >= 2:
    corr = X[numeric_features].corr(numeric_only=True)
    plt.figure(figsize=(6,5)); plt.imshow(corr, aspect='auto'); plt.colorbar(); plt.title('Correlation (numeric)'); plt.show()


## 3. Preprocessing & Split
Handle missing values, encode categorical variables, scale numeric features, and split into train/test

In [ ]:

# Split and create preprocessors
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = [c for c in X.columns if c not in numeric_features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

numeric_transformer = Pipeline([('scaler', StandardScaler(with_mean=False))])
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=True)

preprocessor = ColumnTransformer([('num', numeric_transformer, numeric_features), ('cat', categorical_transformer, categorical_features)], remainder='drop')

print('Preprocessor prepared. Numeric:', len(numeric_features), 'Categorical:', len(categorical_features))


## 4. Model Training & Evaluation
Train Logistic Regression, Random Forest, and SVM. This cell may take a while to run in Colab/local.

In [ ]:

models = {
    'LogisticRegression': LogisticRegression(max_iter=400),
    'RandomForest': RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
    'SVM': SVC(probability=True, random_state=42)
}

results = []
for name, mdl in models.items():
    print('Training', name)
    clf = Pipeline([('preprocess', preprocessor), ('model', mdl)])
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    if hasattr(clf, 'predict_proba'):
        y_proba = clf.predict_proba(X_test)[:,1]
    else:
        if hasattr(clf, 'decision_function'):
            s = clf.decision_function(X_test)
            y_proba = (s - s.min()) / (s.max() - s.min() + 1e-9)
        else:
            y_proba = y_pred.astype(float)
    metrics = {
        'model': name,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_test, y_proba)
    }
    results.append(metrics)
    RocCurveDisplay.from_predictions(y_test, y_proba)
    plt.title('ROC - '+name); plt.show()

import pandas as pd
results_df = pd.DataFrame(results).sort_values('roc_auc', ascending=False)
results_df


## 5. Confusion Matrix & Feature Importance
Fit best model, show confusion matrix and list top features if available.

In [ ]:

best_name = results_df.iloc[0]['model']
print('Best model:', best_name)
best_clf = Pipeline([('preprocess', preprocessor), ('model', models[best_name])])
best_clf.fit(X_train, y_train)
pred = best_clf.predict(X_test)
cm = confusion_matrix(y_test, pred)
print(cm)
plt.imshow(cm); plt.title('Confusion matrix'); plt.colorbar(); plt.show()

# Feature names (optional)
preprocessor.fit(X_train)
feature_names = []
if numeric_features:
    feature_names.extend(numeric_features)
if categorical_features:
    ohe = preprocessor.named_transformers_['cat']
    try:
        ohe_names = ohe.get_feature_names_out(categorical_features).tolist()
    except Exception:
        ohe_names = ohe.get_feature_names(categorical_features).tolist()
    feature_names.extend(ohe_names)

# If classifier supports feature_importances_ or coef_, compute top features
mdl = models[best_name]
mdl.fit(preprocessor.transform(X_train), y_train)
if hasattr(mdl, 'feature_importances_'):
    importances = mdl.feature_importances_
    fi = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False).head(20)
    print(fi)
elif hasattr(mdl, 'coef_'):
    coefs = mdl.coef_
    if getattr(coefs, 'ndim', 1) > 1:
        coefs = coefs[0]
    fi = pd.DataFrame({'feature': feature_names, 'importance': np.abs(coefs)}).sort_values('importance', ascending=False).head(20)
    print(fi)
else:
    print('Feature importance not available for this model.')


## 6. Conclusions & Next Steps
- Interpret the results and consider cross-validation and hyperparameter tuning.
- Save the notebook and export as PDF if required by your instructor.
